## Loading the graphs
We load the graph as a weighted undirected graph.

To load the graph we are using the sister package of Embiggen called [Ensmallen](https://github.com/LucaCappelletti94/ensmallen_graph). Ensmallen is a Rust library with python bindings to handle processing of graph files and preprocessing of data for quickly training embedding models.

In [1]:
from ensmallen_graph import EnsmallenGraph

graph = EnsmallenGraph.from_csv(
    edge_path="/home/jtr4v/merged-kg_edges.tsv",
    sources_column="subject",
    destinations_column="object",
    directed=False,
    edge_types_column="edge_label",
    default_edge_type="biolink:association",
    node_path="/home/jtr4v/merged-kg_nodes.tsv",
    nodes_column="id",
    node_types_column="category",
    default_node_type="biolink:NamedThing",
    ignore_duplicated_edges=True,
    ignore_duplicated_nodes=True,
    force_conversion_to_undirected=True
)

In [2]:
graph.report()

{'density': '0.00021904928054478553',
 'singleton_nodes': '8223',
 'degrees_min': '0',
 'nodes_number': '375365',
 'unique_edge_types_number': '211',
 'edges_number': '30863799',
 'traps_rate': '0.021906677500566116',
 'selfloops_rate': '0.00001539019872440201',
 'unique_node_types_number': '36',
 'bidirectional_rate': '1',
 'degrees_max': '90378',
 'strongly_connected_components_number': '8976',
 'connected_components_number': '8976',
 'degrees_mean': '82.22343319169342',
 'is_directed': 'false',
 'degrees_mode': '1',
 'degrees_median': '6'}

In [3]:
training, validation = graph.connected_holdout(42, 0.8)

In [4]:
training.report()

{'unique_node_types_number': '36',
 'degrees_min': '0',
 'selfloops_rate': '0.000015066194052579398',
 'bidirectional_rate': '1',
 'connected_components_number': '8976',
 'strongly_connected_components_number': '8976',
 'is_directed': 'false',
 'nodes_number': '375365',
 'degrees_mean': '65.77874868461365',
 'degrees_mode': '1',
 'degrees_median': '5',
 'density': '0.0001752394301136591',
 'unique_edge_types_number': '211',
 'degrees_max': '71815',
 'edges_number': '24691040',
 'traps_rate': '0.021927990089646077',
 'singleton_nodes': '8231'}

In [5]:
validation.report()

{'connected_components_number': '162436',
 'degrees_mode': '0',
 'degrees_max': '18563',
 'nodes_number': '375365',
 'singleton_nodes': '158026',
 'degrees_median': '1',
 'unique_edge_types_number': '211',
 'degrees_mean': '16.444684507079774',
 'bidirectional_rate': '1',
 'traps_rate': '0.42099290024376274',
 'strongly_connected_components_number': '162436',
 'selfloops_rate': '0.000016686217621650222',
 'edges_number': '6172759',
 'is_directed': 'false',
 'unique_node_types_number': '36',
 'density': '0.00004380985043112644',
 'degrees_min': '0'}

The followings are check that are not necessary, but are offered as sanity checks:

In [6]:
assert graph > training
assert graph > validation

## Loading the embeddings

In [8]:
import numpy as np

embeddings = np.load("/home/jtr4v/kg_covid_19_drug_analyses/SkipGram_embedding.npy")

In [9]:
node_names = list(np.array(training.nodes_reverse_mapping))

In [10]:
assert len(training.nodes_reverse_mapping) == len(embeddings)

In [11]:
## Rank ChEMBL antivirals by cosine sim to SARS-CoV-2

In [12]:
sars_cov_2_name = 'CHEMBL.TARGET:CHEMBL4303835'

In [13]:
sars_cov_2_idx = node_names.index(sars_cov_2_name)

In [14]:
with open('/home/jtr4v/sars_cov_2_compounds.txt') as f:
    chembl_antiviral_names = f.read().splitlines()

In [15]:
chembl_antiviral_idx = [node_names.index(av) for av in chembl_antiviral_names]

In [16]:
sars_cov_2_emb = embeddings[sars_cov_2_idx]
sars_cov_2_emb.shape

antiviral_emb = embeddings[chembl_antiviral_idx[0]]
antiviral_emb.shape

(100,)

In [17]:
from scipy import spatial
1 - spatial.distance.cosine(antiviral_emb, sars_cov_2_emb)

0.9491932392120361

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

chembl_antiviral_cosine_sim = []

sars_cov_2_emb = embeddings[sars_cov_2_idx]

for antiviral_idx in tqdm(chembl_antiviral_idx):
    antiviral_emb = embeddings[antiviral_idx]
    chembl_antiviral_cosine_sim.append(1 - spatial.distance.cosine(antiviral_emb, sars_cov_2_emb))

100%|██████████| 6900/6900 [00:00<00:00, 21135.57it/s]


In [19]:
chembl_antiviral_names_ranked = [[x,y] for y, x in sorted(zip(chembl_antiviral_cosine_sim,chembl_antiviral_names), key=lambda pair: pair[0], reverse=True)]

In [35]:
import csv

with open("chembl_antiviral_names_ranked.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(chembl_antiviral_names_ranked)

In [20]:
chembl_antiviral_names_ranked

[['CHEMBL.COMPOUND:CHEMBL58', 0.9827765822410583],
 ['CHEMBL.COMPOUND:CHEMBL289277', 0.9827749133110046],
 ['CHEMBL.COMPOUND:CHEMBL297453', 0.9827325344085693],
 ['CHEMBL.COMPOUND:CHEMBL1484', 0.9821076989173889],
 ['CHEMBL.COMPOUND:CHEMBL205596', 0.981408417224884],
 ['CHEMBL.COMPOUND:CHEMBL226335', 0.9810521602630615],
 ['CHEMBL.COMPOUND:CHEMBL224867', 0.980166494846344],
 ['CHEMBL.COMPOUND:CHEMBL145', 0.9799341559410095],
 ['CHEMBL.COMPOUND:CHEMBL1464', 0.9798558950424194],
 ['CHEMBL.COMPOUND:CHEMBL58510', 0.9797195196151733],
 ['CHEMBL.COMPOUND:CHEMBL113', 0.9795886278152466],
 ['CHEMBL.COMPOUND:CHEMBL1069', 0.9792501926422119],
 ['CHEMBL.COMPOUND:CHEMBL104', 0.9789470434188843],
 ['CHEMBL.COMPOUND:CHEMBL130', 0.9787477254867554],
 ['CHEMBL.COMPOUND:CHEMBL719', 0.978517472743988],
 ['CHEMBL.COMPOUND:CHEMBL590799', 0.9782263040542603],
 ['CHEMBL.COMPOUND:CHEMBL939', 0.9780938625335693],
 ['CHEMBL.COMPOUND:CHEMBL1491', 0.9778425693511963],
 ['CHEMBL.COMPOUND:CHEMBL1489', 0.9777946472

In [21]:
chembl_antiviral_names_ranked[0:10]

[['CHEMBL.COMPOUND:CHEMBL58', 0.9827765822410583],
 ['CHEMBL.COMPOUND:CHEMBL289277', 0.9827749133110046],
 ['CHEMBL.COMPOUND:CHEMBL297453', 0.9827325344085693],
 ['CHEMBL.COMPOUND:CHEMBL1484', 0.9821076989173889],
 ['CHEMBL.COMPOUND:CHEMBL205596', 0.981408417224884],
 ['CHEMBL.COMPOUND:CHEMBL226335', 0.9810521602630615],
 ['CHEMBL.COMPOUND:CHEMBL224867', 0.980166494846344],
 ['CHEMBL.COMPOUND:CHEMBL145', 0.9799341559410095],
 ['CHEMBL.COMPOUND:CHEMBL1464', 0.9798558950424194],
 ['CHEMBL.COMPOUND:CHEMBL58510', 0.9797195196151733]]

In [22]:
## Rank all drugs by cosine sim to SARS-CoV-2

In [23]:
assert len(training.node_types) == len(embeddings)

In [24]:
drug_idx = list(np.where(training.node_types == training.node_types_reverse_mapping.index('biolink:Drug'))[0])

In [25]:
drug_names = [node_names[i] for i in drug_idx]

In [26]:
drug_cosine_sim = []

sars_cov_2_emb = embeddings[sars_cov_2_idx]

for drug_i in tqdm(drug_idx):
    drug_emb = embeddings[drug_i]
    drug_cosine_sim.append(1 - spatial.distance.cosine(drug_emb, sars_cov_2_emb))

100%|██████████| 32109/32109 [00:01<00:00, 20572.49it/s]


In [27]:
drugs_ranked = [[x,y] for y, x in sorted(zip(drug_cosine_sim,drug_names), key=lambda pair: pair[0], reverse=True)]

In [28]:
drugs_ranked[0:5]

[['ttd.drug:D0A0UL', 0.968268096446991],
 ['ttd.drug:D0K0KR', 0.9675211906433105],
 ['ttd.drug:D0GP1I', 0.9673413038253784],
 ['ttd.drug:D0MM7G', 0.9671435356140137],
 ['ttd.drug:D03ERS', 0.9667762517929077]]

In [37]:
## Sort everything by closeness to SARS-CoV-2

# Sort all the things

In [30]:
sars_cov_2_emb = embeddings[sars_cov_2_idx]

node_sim = []

for idx in tqdm(range(len(node_names))):
    this_emb = embeddings[idx]
    node_sim.append(1 - spatial.distance.cosine(this_emb, sars_cov_2_emb))

100%|██████████| 375365/375365 [00:18<00:00, 20640.53it/s]


In [31]:
everything_ranked = [[x,y] for y, x in sorted(zip(node_sim,node_names), key=lambda pair: pair[0], reverse=True)]

In [38]:
everything_ranked[0:100]

[['CHEMBL.TARGET:CHEMBL4303835', 1.0],
 ['CORD:26762988147d906e873f16c6e03cbb2ad7c46ea1', 0.9859899878501892],
 ['CORD:19e18a7c27512439dac91fc1096937fd03496c13', 0.9857174754142761],
 ['MESH:D018883', 0.9851577877998352],
 ['CORD:e1d54d1ec033522012a4f994961fec6582c17ce6', 0.9846948385238647],
 ['MESH:D008570', 0.9845601320266724],
 ['CORD:PMC7088121', 0.984553337097168],
 ['CORD:b96d1a769045501e7ba79edf624608b6f2fe5b95', 0.9840669631958008],
 ['CORD:PMC7169570', 0.9838549494743347],
 ['CORD:PMC1592083', 0.9837960004806519],
 ['CORD:e28976acad04032bfb355598922564c65ab7e713', 0.9837953448295593],
 ['CHEMBL.COMPOUND:CHEMBL3137343', 0.9837750792503357],
 ['CORD:PMC7162336', 0.9836901426315308],
 ['CHEMBL.COMPOUND:CHEMBL1625750', 0.9835852384567261],
 ['CORD:4a32929dbc801a7e96e900f42f242f93057211ab', 0.9835548996925354],
 ['CORD:16806b7011f228853fde6f0a67cc11bbf4b448f9', 0.9835430383682251],
 ['HP:0003076', 0.9835119843482971],
 ['HP:0001575', 0.9833900332450867],
 ['MESH:D018442', 0.983186